# SemEval-2020 Task 4: Commonsense Validation and Explanation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rlwPIErsvGcYYgM6s_FT9FDn9lcHzvVU?usp=sharing)

# 0. Getting started

In [ ]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/git/fer-tar

/content/gdrive/.shortcut-targets-by-id/1yfuNPQUT_G0CfNtdGxHhLi96giKyswbZ/git/fer-tar


In [ ]:
!pip install -q pytorch_lightning
!pip install -q transformers
!pip install -q sentencepiece

## 0.1. Imports

In [ ]:
import importlib
import src.models
importlib.reload(src.models)

In [ ]:
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader
from my_scripts import data
from src import encoders
from src.models import ElectraClassifier

In [ ]:
BATCH_SIZE = 16
EPOCHS = 15

In [ ]:
train_ds, val_ds, test_ds = data.ComVEDataset.from_data_folder(x_transforms=[encoders.ElectraEncoder()])
train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=len(val_ds))

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
early_stop_callback = EarlyStopping(
   monitor='val_acc',
   min_delta=0.00,
   patience=5,
   verbose=False,
   mode='max'
)

gpus = 1 if torch.cuda.is_available() else 0
model = ElectraClassifier()
trainer = pl.Trainer(max_epochs=EPOCHS, gpus=gpus, progress_bar_refresh_rate=20, callbacks=[EarlyStopping(monitor='val_loss')], checkpoint_callback=False)
trainer.fit(model, train_dataloader, val_dataloader)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name        | Type         | Params
---------------------------------------------
0 | transformer | ElectraModel | 13.5 M
1 | dropout     | Dropout      

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


## 0.2. Getting the data

## 0.3. Preprocessing